In [1]:
import time

import numpy as np
import pandas as pd
import sklearn.datasets, sklearn.metrics, sklearn.model_selection, sklearn.tree

import subprocess, sys

In [2]:
!cd ../ && make print-SOURCES

SOURCES = cpp/criterion.cpp cpp/factories.cpp cpp/gbt.cpp cpp/lltrees.cpp cpp/metrics.cpp cpp/node.cpp cpp/tree.cpp cpp/wrapper.cpp


In [3]:
# !cd ../ && make clean
!cd ../ && make
subprocess.call([sys.executable, "lltrees_python_import_for_debug.py"])

make: Nothing to be done for 'all'.
[[-1.13017873  2.91817356]
 [-0.97972122 -2.12873409]]
[ 159.09771799 -250.870943    165.74006195  110.75202167   63.47351979]
-----------------------------------------
mode :              regression
epochs :            50
learning_rate :     0.1
metric :            mae
criterion :         absolute_error
max_depth :         5
min_leaf_size :     1
verbose :           1
-----------------------------------------
Type of Training Data : float64
Configuration mode : regression
Epoch : 1     Metric Train : 88.354  Metric va : 96.3501 Residuals (mean) : -508.197
Epoch : 2     Metric Train : 81.2018 Metric va : 89.9313 Residuals (mean) : -457.378
Epoch : 3     Metric Train : 74.6979 Metric va : 84.0295 Residuals (mean) : -411.64
Epoch : 4     Metric Train : 68.8526 Metric va : 78.6383 Residuals (mean) : -370.476
Epoch : 5     Metric Train : 63.4943 Metric va : 74.3066 Residuals (mean) : -333.428
Epoch : 6     Metric Train : 58.7904 Metric va : 69.5122 Resid

0

# make_regression

In [4]:
X, Y = sklearn.datasets.make_regression(n_samples=10000, n_features=8, n_informative=5, n_targets=1, noise=1, random_state=42)
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.3, random_state=42)

In [5]:
sys.path.append('/home/alexandre/Desktop/lltrees/build')
import lltrees
conf ={
    'epochs' : 1,
    'learning_rate' : 1,
    'metric' : 'mae', # Possible metrics for the moment : mae, mse
    'max_depth' : 5,
    'min_leaf_size' : 2,
    'criterion' : "absolute_error",  # Possible metrics for the moment : variance, absolute_error
    'verbose' : 1, 
}
my_lltree = lltrees.lltree()
my_lltree.set_conf(conf)
my_lltree.get_conf()

start_time = time.time()
my_lltree.fit(X_train, Y_train, X_test, Y_test)
print("FIT --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
YP = my_lltree.predict(X_test)
print("PREDICT --- %s seconds ---" % (time.time() - start_time))

print("rmse: %.2f" % np.sqrt(sklearn.metrics.mean_squared_error(Y_test,YP)))
print("mae: %.2f" % sklearn.metrics.mean_absolute_error(Y_test,YP))
print("r2: %.2f" % sklearn.metrics.r2_score(Y_test,YP))

-----------------------------------------
mode :              regression
epochs :            1
learning_rate :     1
metric :            mae
criterion :         absolute_error
max_depth :         5
min_leaf_size :     2
verbose :           1
-----------------------------------------
Type of Training Data : float64
Configuration mode : regression
Epoch : 1     Metric Train : 41.5578 Metric va : 44.4617 Residuals (mean) : -10540.4
FIT --- 0.7796192169189453 seconds ---
PREDICT --- 0.00017118453979492188 seconds ---
rmse: 56.42
mae: 44.46
r2: 0.82


In [6]:
my_lltree = sklearn.tree.DecisionTreeRegressor(max_depth = 5, criterion ="absolute_error", random_state = 0)

start_time = time.time()
my_lltree.fit(X_train, Y_train)
print("FIT --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
YP = my_lltree.predict(X_test)
print("PREDICT --- %s seconds ---" % (time.time() - start_time))

print(YP[0:10])
print("rmse: %.2f" % np.sqrt(sklearn.metrics.mean_squared_error(Y_test,YP)))
print("mae: %.2f" % sklearn.metrics.mean_absolute_error(Y_test,YP))
print("r2: %.2f" % sklearn.metrics.r2_score(Y_test,YP))

FIT --- 1.3003618717193604 seconds ---
PREDICT --- 0.00044274330139160156 seconds ---
[-139.05518197  127.32659619  127.7203612   -84.86192235  221.55630356
 -139.05518197  127.7203612   127.7203612  -117.41140576  111.27770126]
rmse: 56.48
mae: 44.44
r2: 0.82


In [ ]:
conf ={
    'epochs' : 50,
    'learning_rate' : 0.1,
    'metric' : 'mae', # Possible metrics for the moment : mae, mse
    'max_depth' : 5,
    'min_leaf_size' : 1,
    'criterion' : "absolute_error",  # Possible metrics for the moment : variance, absolute_error
    'verbose' : 1, 
}
my_lltree = lltrees.lltree()
my_lltree.set_conf(conf)
my_lltree.get_conf()

start_time = time.time()
my_lltree.fit(X_train, Y_train, X_test, Y_test)
print("FIT --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
YP = my_lltree.predict(X_test)
print("PREDICT --- %s seconds ---" % (time.time() - start_time))

print("rmse: %.2f" % np.sqrt(sklearn.metrics.mean_squared_error(Y_test,YP)))
print("mae: %.2f" % sklearn.metrics.mean_absolute_error(Y_test,YP))
print("r2: %.2f" % sklearn.metrics.r2_score(Y_test,YP))

In [12]:
from sklearn.ensemble import GradientBoostingRegressor
my_lltree = GradientBoostingRegressor(learning_rate = 0.1, n_estimators =50, max_depth = 5,
                                 loss = 'absolute_error', criterion = 'squared_error',
                                 random_state = 0)

start_time = time.time()
my_lltree.fit(X_train, Y_train)
print("FIT --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
YP = my_lltree.predict(X_test)
print("PREDICT --- %s seconds ---" % (time.time() - start_time))

print("rmse: %.2f" % np.sqrt(sklearn.metrics.mean_squared_error(Y_test,YP)))
print("mae: %.2f" % sklearn.metrics.mean_absolute_error(Y_test,YP))
print("r2: %.2f" % sklearn.metrics.r2_score(Y_test,YP))

FIT --- 0.29142022132873535 seconds ---
PREDICT --- 0.0005440711975097656 seconds ---
rmse: 0.34
mae: 0.12
r2: 0.54


# make_classification

In [9]:
X, Y = sklearn.datasets.make_classification(n_samples=1000, n_features=8, n_informative=5, n_classes=2, random_state=42)
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.3, random_state=42)

In [10]:
# conf ={
#     'epochs' : 50,
#     'learning_rate' : 0.1,
#     'metric_name' : 'mae', # Possible metrics for the moment : mae, mse
#     'lltree_max_depth' : 3,
#     'lltree_min_size_split' : 1,
#     'lltree_criterion' : "gini",  # Possible metrics for the moment : variance, absolute_error
#     'verbose' : 1,  # Possible metrics for the moment : variance, absolute_error
# }
# my_lltree = lltrees.lltree()
# my_lltree.set_conf(conf)
# my_lltree.get_conf()

# start_time = time.time()
# my_lltree.fit(X_train, Y_train)
# print("FIT --- %s seconds ---" % (time.time() - start_time))

# start_time = time.time()
# YP = my_lltree.predict(X_test)
# print("PREDICT --- %s seconds ---" % (time.time() - start_time))

# print("accuracy_score: %.2f" % np.sqrt(sklearn.metrics.accuracy_score(Y_test,YP)))
# print("log_loss: %.2f" % sklearn.metrics.log_loss(Y_test,YP))
# print("f1_score: %.2f" % sklearn.metrics.f1_score(Y_test,YP))